CIFAR 10 daraset


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Configuring the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setting hyperparameters for training the model
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# Defining the transform to normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Loading the CIFAR10 dataset and applying the transformations
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Creating data loaders for training and testing datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Defining the classes of the CIFAR10 dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:02<00:00, 59590728.41it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Defining the neural network architecture
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        # Defining the layers
        self.conv_layer1 = nn.Conv2d(3, 32, 3)
        self.pooling_layer = nn.MaxPool2d(2, 2)
        self.conv_layer2 = nn.Conv2d(32, 64, 3)
        self.conv_layer3 = nn.Conv2d(64, 64, 3)
        self.fc_layer1 = nn.Linear(64*4*4, 64)
        self.fc_layer2 = nn.Linear(64, 10)

    def forward(self, x):
        # Forward pass through the network
        x = F.relu(self.conv_layer1(x))
        x = self.pooling_layer(x)
        x = F.relu(self.conv_layer2(x))
        x = self.pooling_layer(x)
        x = F.relu(self.conv_layer3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc_layer1(x))
        x = self.fc_layer2(x)
        return x

In [4]:
# Initializing the model, loss function, and optimizer
model = ConvolutionalNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Counting the number of batches per epoch
n_total_steps = len(train_loader)

# Training the model
for epoch in range(num_epochs):
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        # Moving the images and labels to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass to get the outputs
        outputs = model(images)
        # Calculating the loss
        loss = criterion(outputs, labels)

        # Backward pass to update the weights
        loss.backward()
        optimizer.step()
        # Resetting the gradients
        optimizer.zero_grad()

        # Accumulating the loss over the epoch
        running_loss += loss.item()

    # Printing the average loss per epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / n_total_steps:.3f}')

# Saving the trained model
model_save_path = './cnn.pth'
torch.save(model.state_dict(), model_save_path)

Epoch [1/10], Loss: 1.494
Epoch [2/10], Loss: 1.140
Epoch [3/10], Loss: 0.975
Epoch [4/10], Loss: 0.863
Epoch [5/10], Loss: 0.783
Epoch [6/10], Loss: 0.726
Epoch [7/10], Loss: 0.674
Epoch [8/10], Loss: 0.626
Epoch [9/10], Loss: 0.591
Epoch [10/10], Loss: 0.555
